# Copy & Paste code from .py files

In [2]:
# Required imports
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
# Required imports
import ee
import numpy as np

# Required imports
import google
from google.colab import auth
import folium
import requests
import io
from folium import plugins
import numpy as np

In [3]:
# Standard authentication cell
auth.authenticate_user()
credentials, project_id = google.auth.default()
ee.Initialize(credentials, project='semiotic-garden-395711')

In [4]:
# Required imports
import requests
from typing import Iterable

""" Defines the functions used to get the data for initial model training. """

# Remaps the target land classification from mutli-class to binary
def get_target_image(target) -> ee.Image:
    """ Buckets multi-class land cover classifications into 2 classes:
    1 = forest
    0 = non-forest """
    # Remap the ESA classifications into the Dynamic World classifications
    fromValues = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
    toValues = [1, 1, 1, 1, 1, 0, 0, 0,0, 0, 0,0,0,0,0,0,0]
    return (
        target.first()
        .select("LC_Type1")
        .remap(fromValues, toValues)
        .rename("landcover")
        .unmask(0)  # fill missing values with 0 (water)
        .byte()  # 9 classifications fit into an unsinged 8-bit integer
    )

# Getting the target sample points, equally stratified across both classes
def sample_points(
    region: ee.Geometry, image: ee.Image, points_per_class: int, scale: int
) -> Iterable[tuple[float, float]]:
    """ Applies the stratified sampling algorithm to the given target image."""
    points = image.stratifiedSample(
        points_per_class,
        region=region,
        scale=scale,
        geometries=True,
    )
    for point in points.toList(points.size()).getInfo():
        yield point["geometry"]["coordinates"]

# OBSOLETE
# Getting the coordinates for the target points using Ana's random 100 approach
def get_coordinates(points):
    """ Returns a dictionary of square pixel coordinates for the target points. """
    target_dict = {}
    numb = 1

    # Iterating through the global image to generate stratified sampling coordinates
    for point in points:
        target_dict[f"P{numb}"] = list(point)
        numb +=1

    return target_dict

# TO DO - make this more robust. it works for about 80 points but can break past 100
# Getting the coordinates for the target points using Felix' stratified approach
def get_coordinates_felix(polygon, target):
    """ Returns a dictionary of square pixel coordinates for the target points. """
    # Defining the region of interest
    region = ee.Geometry.Polygon(polygon)

    # Getting the target image and creating a dictionary to store the coordinates
    labels_image = get_target_image(target)
    target_dict = {}
    numb = 1

    # Iterating through the global image to generate stratified sampling coordinates
    for point in sample_points(region, labels_image, points_per_class=2, scale=500):
        target_dict[f"P{numb}"] = point
        numb +=1

    return target_dict

# Extracting the coordinates

# Taking the coordinates and getting the features data for the target points
def get_data(target_dict, year, feature_bands, target):
    """ Get the feature and target data, both as ndarrays. """

    ### FEATURES ###

    # Initialize an empty list to hold the images and skipped points
    stacked_feature_list = []
    skipped_points = []

    # Debugging counter for featuress
    features_counter = 0

    # Loop over each year from year
    for point in target_dict:
        # Get the picked point and create a 500m x 500m square around it
        picked_point = ee.Geometry.Point(target_dict[point])
        square = picked_point.buffer(250).bounds()

        # Define image collection features
        image_collection_features = (ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
            .filterDate(f"{year}-1-1", f"{year+3}-12-31")
            .filterBounds(square)
            .select(feature_bands)
            .sort('system:time_start'))

        # Check size of the image collection
        count = image_collection_features.size().getInfo()
        if count == 0:
            print(f"Skipping point: {point}")
            skipped_points.append(point)
            continue

        # Get the first image
        image_features = image_collection_features.first()

        # Clip the gotten image to the 500m x 500m square
        c_img_features = image_features.clip(square)

        # Get the download url for the clipped image
        url = c_img_features.getDownloadUrl({
            'scale': 10, # Because the feature satellite images are 10m x 10m per pixel in resolution
            'format': 'NPY' # numpy
            })

        # Get the image as a numpy array
        image_array_features = requests.get(url)
        image_array_features = np.load(io.BytesIO(image_array_features.content))

        # Creating the NDVI array - NDVI is an index used for detecting forest in the academic literature
        # Extract B4 (Red) and B8 (NIR)
        B4 = image_array_features['B4'].astype(float)
        B8 = image_array_features['B8'].astype(float)

        # Calculate NDVI - basically the normalised difference between Red and NIR bands
        NDVI = (B8 - B4) / (B8 + B4 + 1e-10)  # adding a small constant to avoid division by zero

        # Append the numpy array to the list
        stacked_feature_list.append(NDVI)
        print(stacked_feature_list[-1]) # Uncomment to see the numpy array
        print(f"Appending feature {features_counter} with shape {NDVI.shape}") # Uncomment to see the shape of the numpy array
        features_counter += 1

    # Apply cropping to the numpy array to ensure consistent shape
    cropped_arrays_features = []

    for arr in stacked_feature_list:
        cropped_features = arr[:50, :50]
        cropped_arrays_features.append(cropped_features)

    feature_stacked_array = np.stack(cropped_arrays_features, axis=0)

    ### TARGETS ###

    # Account for the fact that some points were skipped in feature dataset, and we must maintain matching target points that remain
    new_target_dict = {k: v for k, v in target_dict.items() if k not in skipped_points}

    # Initialize an empty list to hold the images and skipped points
    stacked_target_list = []

    # Debugging counter for targets
    target_counter = 0

    # Loop over each year from year
    for point in new_target_dict:
        # Get the picked point and create a 500m x 500m square around it
        picked_point = ee.Geometry.Point(target_dict[point])
        square = picked_point.buffer(250).bounds()

        # Clip the gotten image to the 500m x 500m square
        c_img_target = target.clip(square)

        # Get the download url for the clipped image
        url = c_img_target.getDownloadUrl({
            'scale': 500, # Because the target satellite images are 500m x 500m per pixel in resolution
            'format': 'NPY' # numpy
            })

        # Get the image as a numpy array
        image_array_targets = requests.get(url)
        image_array_targets = np.load(io.BytesIO(image_array_targets.content))

        # Append the numpy array to the list
        stacked_target_list.append(image_array_targets)
        print(stacked_target_list[-1]) # Uncomment to see the numpy array
        print(f"Appending target {target_counter} with shape {image_array_targets.shape}") # Uncomment to see the shape of the numpy array
        target_counter += 1

    # Apply cropping to the numpy array to ensure consistent shape
    cropped_arrays_targets = []

    for arr in stacked_target_list:
        cropped_targets = arr[:3, :3]
        cropped_arrays_targets.append(cropped_targets)

    target_stacked_array = np.stack(cropped_arrays_targets, axis=0)

    ### TRAINING AND TEST DATASETS ###

    # Separating the feature and target arrays into training and test datasets using 80/20 split

    depth = target_stacked_array.shape[0]
    split_index = int(depth * 0.8)  # 80% for training

    # Train-test split
    train_target = target_stacked_array[:split_index]
    test_target = target_stacked_array[split_index:]

    train_feature = feature_stacked_array[:split_index]
    test_feature = feature_stacked_array[split_index:]
    print(train_feature.shape, train_target.shape, test_feature.shape, test_target.shape)
    return train_feature, train_target, test_feature, test_target


In [5]:

""" Provides the setpoint values according to which the data will be collected. """

# # Initialise the Earth Engine module.
# ee.Initialize()

# Defining the main year around which data will be collected
f_date = '2017'

# Defining the target ImageCollection, filtered by the main year
target = (ee.ImageCollection("MODIS/061/MCD12Q1")
          .filterDate(f_date)
          .sort('system:time_start'))  # Sort by time to get earliest image

# Oversimplified North America region.
polygon = [[[-145.7, 63.2], [-118.1, 22.3], [-78.2, 5.6], [-52.9, 47.6]]]

# Global polygon, while minimising the amount of water
#polygon = [[[-180, -60], [180, -60], [180, 85], [-180, 85], [-180, -60]]]

# Select the feature bands
feature_bands = ["B4", "B8"]

# Running the function get_coordinates to test the script
get_data(get_coordinates_felix(polygon, target), int(f_date), feature_bands, get_target_image(target))


[[0.         0.         0.         ... 0.02518451 0.         0.        ]
 [0.02650442 0.04333438 0.02870115 ... 0.02841443 0.         0.        ]
 [0.03466287 0.03767661 0.03633697 ... 0.02888828 0.         0.        ]
 ...
 [0.         0.         0.06868638 ... 0.05282274 0.04320747 0.        ]
 [0.         0.         0.06079976 ... 0.03807068 0.03906546 0.        ]
 [0.         0.         0.04646775 ... 0.         0.         0.        ]]
Appending feature 0 with shape (52, 53)
[[ 0.         -0.08589744 -0.10251451 ...  0.          0.
   0.        ]
 [ 0.         -0.10077519 -0.10707333 ... -0.09554974 -0.09920635
  -0.06961614]
 [ 0.         -0.1092437  -0.09639344 ... -0.08996089 -0.05905256
  -0.06970509]
 ...
 [ 0.         -0.06565657 -0.06102117 ... -0.08894536 -0.08881789
   0.        ]
 [ 0.         -0.07228158 -0.09011809 ... -0.08359923 -0.09102564
   0.        ]
 [ 0.          0.          0.         ... -0.07594129 -0.08007687
   0.        ]]
Appending feature 1 with shape (

(array([[[ 0.        ,  0.        ,  0.        , ...,  0.02173291,
           0.02023879,  0.02179392],
         [ 0.02650442,  0.04333438,  0.02870115, ...,  0.02335746,
           0.02036532,  0.02074043],
         [ 0.03466287,  0.03767661,  0.03633697, ...,  0.02715171,
           0.02129933,  0.02142616],
         ...,
         [ 0.        ,  0.        ,  0.08288791, ...,  0.02973736,
           0.02871025,  0.04002982],
         [ 0.        ,  0.        ,  0.07696074, ...,  0.02606466,
           0.02642676,  0.03320599],
         [ 0.        ,  0.        ,  0.06868638, ...,  0.02209172,
           0.02735605,  0.03792798]],
 
        [[ 0.        , -0.08589744, -0.10251451, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        , -0.10077519, -0.10707333, ..., -0.10919921,
          -0.09079118, -0.09554974],
         [ 0.        , -0.1092437 , -0.09639344, ..., -0.11477573,
          -0.09520683, -0.08996089],
         ...,
         [ 0.        , -0.0591195

In [6]:
# Required imports
import numpy as np
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers

# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam, Nadam
# from tensorflow.keras import applications, optimizers
# from tensorflow.keras.applications import InceptionResNetV2
# from tensorflow.keras.applications.resnet50 import preprocess_input

# from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
# from tensorflow.keras.utils import model_to_dot, plot_model, image_dataset_from_directory
# from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, LearningRateScheduler
# from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, ZeroPadding2D, Dropout

from sklearn.metrics import f1_score



# Defining the majority pooling function for the baseline model
def majority_pool(array):
    dt = np.dtype([("ndvi", np.int32)])
    pooled_array = np.zeros((array.shape[0], 3, 3), dtype=dt)

    step_x = step_y = 16

    for n in range(array.shape[0]):
        for i in range(3):
            x_start = i * step_x
            for j in range(3):
                y_start = j * step_y
                quadrant = array[n, x_start:x_start+step_x, y_start:y_start+step_y]
                majority = np.sum(quadrant) > (step_x * step_y // 2)
                pooled_array[n, i, j]['ndvi'] = int(majority)

    return pooled_array

# Define the baseline model
def baseline(test_feature):
    """ Baseline model that uses majority pooling on a threshold NDVI value. """
    mask_ndvi = test_feature >= 0.6
    NDVI_bucketed = np.where(mask_ndvi, 1, 0)
    y_pred_baseline = majority_pool(NDVI_bucketed)
    return y_pred_baseline

#Convert a structured ndarray to a standard ndarray and expand dimensions to align with CNN input.
def process_and_expand(structured_array,  dtype=np.float32):
    """
    Parameters:
        structured_array (numpy.ndarray): The structured array to convert.
        field_name (str): The name of the field to extract from the structured array.
        dtype (numpy.dtype, optional): The desired dtype for the output array. Defaults to np.float32.

    Returns:
        numpy.ndarray: The converted and expanded standard ndarray.
    """
    standard_array = np.array(structured_array, dtype=dtype)
    expanded_array = np.expand_dims(standard_array, axis=-1)
    return expanded_array


# Define and train the CNN model
def train_cnn(train_feature_expanded, train_target_expanded):

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_feature_expanded, train_target_expanded, epochs=10, batch_size=32, validation_split=0.2)

    return model

# Run a prediction from the CNN model
def predict_cnn(model, test_feature):
    predictions = model.predict(test_feature)
    # Round predictions to get binary classification output
    rounded_predictions = [round(x[0]) for x in predictions]
    return rounded_predictions

# predictions = predict_cnn(trained_model, test_feature_reshaped)
# print("Predictions:", predictions)

# Define the evaluation function
def evaluate(test_target, y_pred):
    """ Evaluates the model using the F1 score. """
    # Reshape your arrays into 1D arrays
    #true_values_1D = test_target.reshape(-1)
    #pred_values_1D = y_pred.reshape(-1)

    true_values_1D = test_target["landcover"].flatten()
    pred_values_1D = y_pred['ndvi'].flatten()

    # Calculate F1 score
    f1 = f1_score(true_values_1D, pred_values_1D)

    print("F1 Score:", f1)


In [7]:
train_f, train_t, test_f, test_t = get_data(get_coordinates_felix(polygon, target), int(f_date), feature_bands, get_target_image(target))

processed_train_f = process_and_expand(train_f)
processed_train_t = process_and_expand(train_t)
processed_test_f = process_and_expand(test_f)
processed_test_t = process_and_expand(test_t)


[[0.         0.         0.         ... 0.02518451 0.         0.        ]
 [0.02650442 0.04333438 0.02870115 ... 0.02841443 0.         0.        ]
 [0.03466287 0.03767661 0.03633697 ... 0.02888828 0.         0.        ]
 ...
 [0.         0.         0.06868638 ... 0.05282274 0.04320747 0.        ]
 [0.         0.         0.06079976 ... 0.03807068 0.03906546 0.        ]
 [0.         0.         0.04646775 ... 0.         0.         0.        ]]
Appending feature 0 with shape (52, 53)
[[ 0.         -0.08589744 -0.10251451 ...  0.          0.
   0.        ]
 [ 0.         -0.10077519 -0.10707333 ... -0.09554974 -0.09920635
  -0.06961614]
 [ 0.         -0.1092437  -0.09639344 ... -0.08996089 -0.05905256
  -0.06970509]
 ...
 [ 0.         -0.06565657 -0.06102117 ... -0.08894536 -0.08881789
   0.        ]
 [ 0.         -0.07228158 -0.09011809 ... -0.08359923 -0.09102564
   0.        ]
 [ 0.          0.          0.         ... -0.07594129 -0.08007687
   0.        ]]
Appending feature 1 with shape (

# Model

In [12]:
# Required imports
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, ZeroPadding2D, Dropout, MaxPooling2D, Flatten, Dense, Reshape
from tensorflow.keras.metrics import Metric
import tensorflow as tf

class F1Score(Metric):

    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)
        values = tf.cast(y_true, tf.float32)
        predictions = tf.cast(y_pred, tf.float32)
        self.true_positives.assign_add(tf.reduce_sum(values * predictions))
        self.false_positives.assign_add(tf.reduce_sum((1.0 - values) * predictions))
        self.false_negatives.assign_add(tf.reduce_sum(values * (1.0 - predictions)))

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))



input_img = tf.keras.layers.Input(shape=(50, 50, 1))

# Convolutional Block 1
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = BatchNormalization()(x) # Add batch normalization for stability
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.25)(x) # Add dropout for regularization

# Convolutional Block 2
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.25)(x)

# Convolutional Block 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.25)(x)

# Flattening the feature map
x = Flatten()(x)

# Dense layers
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Dense layer to get the desired number of features
x = Dense(3*3*1, activation='sigmoid')(x)

# Reshaping to the target shape
decoded = Reshape((3, 3, 1))(x)

# Create the model
model = tf.keras.models.Model(input_img, decoded)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[F1Score()])




# Fit the model
history = model.fit(
    processed_train_f,
    processed_train_t,
    epochs=100,
    batch_size=32,
    shuffle=True
)


Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.4618 - f1_score: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 83ms/step - loss: 0.3587 - f1_score: 0.1429
Epoch 3/100
1/1 [==============================] - 0s 92ms/step - loss: 0.3707 - f1_score: 0.2500
Epoch 4/100
1/1 [==============================] - 0s 85ms/step - loss: 0.4467 - f1_score: 0.1176
Epoch 5/100
1/1 [==============================] - 0s 86ms/step - loss: 0.2031 - f1_score: 0.4000
Epoch 6/100
1/1 [==============================] - 0s 115ms/step - loss: 0.2127 - f1_score: 0.4000
Epoch 7/100
1/1 [==============================] - 0s 87ms/step - loss: 0.1682 - f1_score: 0.4444
Epoch 8/100
1/1 [==============================] - 0s 87ms/step - loss: 0.4010 - f1_score: 0.1429
Epoch 9/100
1/1 [==============================] - 0s 100ms/step - loss: 0.1997 - f1_score: 0.2500
Epoch 10/100
1/1 [==============================] - 0s 90ms/step - loss: 0.1549 - f1_score: 0.0000e+00
Epoch 11/10

In [14]:
# y_pred = model.predict(processed_test_f)

In [13]:
loss, f1_score = model.evaluate(processed_test_f, processed_test_t)
print("Test Loss:", loss)
print("Test F1 Score:", f1_score)

1/1 [==============================] - 0s 291ms/step - loss: 0.2222 - f1_score: 0.0000e+00
Test Loss: 0.2222222238779068
Test F1 Score: 0.0
